<a href="https://colab.research.google.com/github/Junaidkhan1485786/pincone_chatbot_agent-/blob/main/chatbot_agent_by_pincone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU \
    openai==0.27.7 \
    pinecone-client==3.1.0 \
    pinecone-datasets==0.7.0 \
    langchain==0.1.1 \
    langchain-community==0.0.13 \
    tiktoken==0.4.0 \
    pinecone-notebooks==0.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49

In [2]:
from pinecone_datasets import load_dataset

dataset = load_dataset("squad-text-embedding-ada-002")
dataset.head()

,id,values,sparse_values,metadata,blob
0,5733be284776f41900661182,"[-0.010262451963272523, 0.02222637996192584, -...",None,"{'text': 'Architecturally, the school has a Ca...",None
1,5733bf84d058e614000b61be,"[-0.009786712423983223, -0.013988726438873078,...",None,"{'text': 'As at most other universities, Notre...",None
2,5733bed24776f41900661188,"[0.013343917696606181, -0.0007001232846109822,...",None,{'text': 'The university is the major seat of ...,None
3,5733a6424776f41900660f51,"[-0.0085222901071539, 0.004399558219521822, -0...",None,{'text': 'The College of Engineering was estab...,None
4,5733a70c4776f41900660f64,"[-0.006695996885869355, -0.02067068565761649, ...",None,{'text': 'All of Notre Dame's undergraduate st...,None


In [6]:
print(dataset)
len(dataset)

18891

In [7]:
# we drop sparse_values as they are not needed for this example
dataset.documents.drop(['sparse_values', 'blob'], axis=1, inplace=True)

dataset.head()

,id,values,metadata
0,5733be284776f41900661182,"[-0.010262451963272523, 0.02222637996192584, -...","{'text': 'Architecturally, the school has a Ca..."
1,5733bf84d058e614000b61be,"[-0.009786712423983223, -0.013988726438873078,...","{'text': 'As at most other universities, Notre..."
2,5733bed24776f41900661188,"[0.013343917696606181, -0.0007001232846109822,...",{'text': 'The university is the major seat of ...
3,5733a6424776f41900660f51,"[-0.0085222901071539, 0.004399558219521822, -0...",{'text': 'The College of Engineering was estab...
4,5733a70c4776f41900660f64,"[-0.006695996885869355, -0.02067068565761649, ...",{'text': 'All of Notre Dame's undergraduate st...


In [8]:
import os

if not os.environ.get("pincone key enter here"):
    from pinecone_notebooks.colab import Authenticate
    Authenticate()

In [9]:
from pinecone import Pinecone

api_key = os.environ.get("pincone key enter here")

# configure client
pc = Pinecone(api_key=api_key)

In [10]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [11]:
index_name = 'langchain-retrieval-agent-fast'

In [12]:
import time

if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

# we create a new index
pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of text-embedding-ada-002
        metric='dotproduct',
        spec=spec
    )

# wait for index to be initialized
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

In [13]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [14]:
index.upsert_from_dataframe(dataset.documents, batch_size=100)

sending upsert requests:   0%|          | 0/18891 [00:00<?, ?it/s]

{'upserted_count': 18891}

In [15]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 19091}},
 'total_vector_count': 19091}

In [ ]:
# Creating a Vector Store and Querying

from langchain.embeddings.openai import OpenAIEmbeddings

openai_api_key = os.environ.get('OPENAI_API_KEY') or 'OPENAI_API_KEY'
model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=openai_api_key
)

In [ ]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pc.Index(index_name)

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

In [ ]:
query = "when was the college of engineering in the University of Notre Dame established?"

vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

In [ ]:
# Initializing the Conversational Agent

from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

# chat completion llm
llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)
# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [ ]:
from langchain.agents import Tool

tools = [
    Tool(
        name='Knowledge Base',
        func=qa.run,
        description=(
            'use this tool when answering general knowledge queries to get '
            'more information about the topic'
        )
    )
]

In [ ]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

In [ ]:
agent(query)

# output


# > Entering new AgentExecutor chain...
# {
#     "action": "Knowledge Base",
#     "action_input": "When was the College of Engineering in the University of Notre Dame established?"
# }
# Observation: The College of Engineering at the University of Notre Dame was established in 1920.
# Thought:{
#     "action": "Final Answer",
#     "action_input": "The College of Engineering at the University of Notre Dame was established in 1920."
# }

# > Finished chain.
# {'input': 'when was the college of engineering in the University of Notre Dame established?',
#  'chat_history': [],
#  'output': 'The College of Engineering at the University of Notre Dame was established in 1920.'}

In [ ]:
pc.delete_index(index_name)